# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [135]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [136]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 

## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [137]:
# frequency table for prestige and whether or not someone was admitted
print df["prestige"].value_counts()

2.0    148
3.0    121
4.0     67
1.0     61
Name: prestige, dtype: int64


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [138]:

##df = df.join pd.get_dummies(df['prestige'])  # some reason its gets error overlap when i try to run twice

dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige')
print dummy_ranks.head()
##print dummy_ranks.info()
##print df.describe()

   prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0           0.0           0.0           1.0           0.0
1           0.0           0.0           1.0           0.0
2           1.0           0.0           0.0           0.0
3           0.0           0.0           0.0           1.0
4           0.0           0.0           0.0           1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: When modeling class variables we should have try to use lines to draw lines through where data can be separated off.Typically we would at least 2+ otherwise the relationship would be better understood without creating classes.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [139]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           0.0           1.0           0.0
1      1  660.0  3.67           0.0           0.0           1.0           0.0
2      1  800.0  4.00           1.0           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           0.0           1.0


In [140]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
print pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])



prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [141]:
#print df.prestige.value_counts()
#pres1prob= sum(df['prestige(1.0)'])

odds_pres_1 = .54/(1-.54) 
print odds_pres_1

1.17391304348


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [142]:
odds_others = .28/(1-.28)
print odds_others

0.388888888889


#### 3.3 Calculate the odds ratio

In [143]:
print "odds ratio " + str(odds_pres_1/odds_others)

odds ratio 3.01863354037


#### 3.4 Write this finding in a sentenance: 

Answer:  attanding an undergraduate school with a prestige score as 1 over all had nearly a double percent lift in admittance.

#### 3.5 Print the cross tab for prestige_4

In [144]:
print pd.crosstab(df['admit'], df['prestige'], rownames=['admit'])



prestige  1.0  2.0  3.0  4.0
admit                       
0          28   95   93   55
1          33   53   28   12


#### 3.6 Calculate the OR 

In [145]:

12./55

0.21818181818181817

#### 3.7 Write this finding in a sentence

Answer: the likelyhood of you being admitted is higher for students who attended schools wth a higher prestige

## Part 4. Analysis

In [165]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           1.0           0.0
1      1  660.0  3.67           0.0           1.0           0.0
2      1  800.0  4.00           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [160]:
# manually add the intercept
data['intercept'] = 1.0
print data.head()

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0  intercept
0      0  380.0  3.61           0.0           1.0           0.0        1.0
1      1  660.0  3.67           0.0           1.0           0.0        1.0
2      1  800.0  4.00           0.0           0.0           0.0        1.0
3      1  640.0  3.19           0.0           0.0           1.0        1.0
4      0  520.0  2.93           0.0           0.0           1.0        1.0


#### 4.1 Set the covariates to a variable called train_cols

In [148]:
train_cols = data.columns[1:]


#### 4.2 Fit the model

In [149]:
logit = sm.Logit(data['admit'], data[train_cols])
result = logit.fit()


Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [150]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Thu, 20 Oct 2016   Pseudo R-squ.:                 0.08166
Time:                        18:24:10   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [151]:
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer  as prestige 2 goes up by 1 the log odds goes up by 

#### 4.6 Interpret the OR of GPA

Answer: as gpa goes up by 1 the log odds goes down by .8

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [152]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [153]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))
 


[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [170]:
# recreate the dummy variables

# keep only what we need for making predictions
#print combos.head()
dummy_rank2 = pd.get_dummies(combos[2], prefix=2)
cols_to_keep2 = [0,1,3]
data3 = combos[cols_to_keep2].join(dummy_rank2.ix[:,'2_2.0':])
print data3.head()


       0         1    3  2_2.0  2_3.0  2_4.0
0  220.0  2.260000  1.0    0.0    0.0    0.0
1  220.0  2.260000  1.0    1.0    0.0    0.0
2  220.0  2.260000  1.0    0.0    1.0    0.0
3  220.0  2.260000  1.0    0.0    0.0    1.0
4  220.0  2.453333  1.0    0.0    0.0    0.0


#### 5.2 Make predictions on the enumerated dataset

In [171]:
result.predict(data3)

array([ 0.82767157,  0.55738036,  0.50395086,  0.09049221,  0.84811417,
        0.59416465,  0.54152305,  0.10368189,  0.86652283,  0.62992167,
        0.57862892,  0.11854346,  0.88300812,  0.66430818,  0.6148675 ,
        0.1352139 ,  0.89769775,  0.69703631,  0.64987532,  0.15381956,
        0.9107294 ,  0.72788066,  0.68333982,  0.17446889,  0.92224481,
        0.75668097,  0.71500847,  0.1972441 ,  0.93238508,  0.78334044,
        0.74469328,  0.22219218,  0.94128712,  0.8078208 ,  0.77227091,
        0.24931576,  0.94908115,  0.83013493,  0.79767897,  0.27856456,
        0.84711857,  0.59230463,  0.53960871,  0.10296774,  0.86562878,
        0.62812296,  0.57674842,  0.1177404 ,  0.88220948,  0.66258709,
        0.61304057,  0.13431512,  0.89698769,  0.69540607,  0.64811942,
        0.15281896,  0.91010073,  0.72635123,  0.68166954,  0.17336148,
        0.92169027,  0.75525899,  0.71343518,  0.19602646,  0.93189752,
        0.78202938,  0.74322502,  0.2208629 ,  0.94085968,  0.80

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.